# Бэггинг и градиентный бустинг

## Бэггинг и градиентный бустинг.

Давайте проанализируем, как производительность бэггинга и градиентного бустинга зависит от количества базовых элементов в ансамбле.

В случае бэггинга все базовые модели подгоняются под разные выборки из одного и того же распределения данных $\mathbb{X} \times \mathbb{Y}$. Некоторые из них могут быть переобучены, тем не менее, последующее усреднение их предсказаний позволяет смягчить этот эффект. 

Причина этого в том, что для некоррелированных алгоритмов дисперсия их композиции в $N$ раз ниже индивидуальной. Другими словами, крайне маловероятно, что все компоненты ансамбля переобучаются под какой-то нетипичный объект из обучающего набора (по сравнению с одной моделью). Когда размер ансамбля $N$ становится достаточно большим, дальнейшее добавление базовых моделей не увеличивает качество.

При бустинге каждый алгоритм подгоняется под ошибки текущей построенной композиции, что позволяет ансамблю постепенно улучшать качество аппроксимации распределения данных. Однако увеличение размера ансамбля $N$ может привести к переобучению, поскольку добавление новых моделей в композицию еще больше соответствует обучающим данным и в конечном итоге может снизить способность к обобщению.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
X_train = np.linspace(0, 1, 100)
X_test = np.linspace(0, 1, 1000)

@np.vectorize
def target(x):
    return x > 0.5

Y_train = target(X_train) + np.random.randn(*X_train.shape) * 0.1

plt.figure(figsize = (8, 6))
plt.scatter(X_train, Y_train, s=50)
plt.show()

Для начала возьмем алгоритм бэггинга на деревьях решений.

Здесь размер ансамбля постепенно увеличивается.
Давайте посмотрим, как прогноз зависит от размера.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor

reg = BaggingRegressor(DecisionTreeRegressor(max_depth=2), warm_start=True)
plt.figure(figsize=(15, 12))
sizes = [1, 2, 5, 20, 100, 500, 1000, 2000]
for i, s in enumerate(sizes):
    reg.n_estimators = s
    reg.fit(X_train.reshape(-1, 1), Y_train)
    plt.subplot(4, 2, i+1)
    plt.xlim([0, 1])
    plt.scatter(X_train, Y_train, s=10)
    plt.plot(X_test, reg.predict(X_test.reshape(-1, 1)), c='green', linewidth=2)
    plt.title('{} trees'.format(s))

Можете видеть, что после определенного момента общий прогноз не меняется при добавлении базовых моделей.

Теперь давайте сделаем то же самое с градиентным бустингом.

In [ ]:
reg = GradientBoostingRegressor(max_depth=1, learning_rate=1, warm_start=True)
plt.figure(figsize=(15, 12))
sizes = [1, 2, 5, 20, 100, 500, 1000, 2000]
for i, s in enumerate(sizes):
    reg.n_estimators = s
    reg.fit(X_train.reshape(-1, 1), Y_train)
    plt.subplot(4, 2, i+1)
    plt.xlim([0, 1])
    plt.scatter(X_train, Y_train, s=10)
    plt.plot(X_test, reg.predict(X_test.reshape(-1, 1)), c='green', linewidth=2)
    plt.title('{} trees'.format(s))

Градиентный бустинг быстро улавливает истинную зависимость, но затем начинает переобучение по отношению к отдельным объектам из обучающего набора. В результате модели с большими размерами ансамбля оказываются сильно переобученными.

Можно решить эту проблему, выбрав очень простую базовый модель или намеренно снизив вес последующих алгоритмов в композиции:
$$
a_N(x) = \sum_{n=0}^N \eta \gamma_N b_n(x).
$$

Здесь $\eta$ — это параметр шага, который контролирует влияние новых компонентов ансамбля.

Такой подход делает обучение медленнее по сравнению с бэггингом, однако делает конечную модель менее переобученной. Тем не менее следует помнить, что переобучение может произойти для любого $\eta$ в пределе бесконечного размера ансамбля.

In [ ]:
reg = GradientBoostingRegressor(max_depth=1, learning_rate=0.1, warm_start=True)
plt.figure(figsize=(15, 12))
sizes = [1, 2, 5, 20, 100, 500, 1000, 2000]
for i, s in enumerate(sizes):
    reg.n_estimators = s
    reg.fit(X_train.reshape(-1, 1), Y_train)
    plt.subplot(4, 2, i+1)
    plt.xlim([0, 1])
    plt.scatter(X_train, Y_train, s=10)
    plt.plot(X_test, reg.predict(X_test.reshape(-1, 1)), c='green', linewidth=2)
    plt.title('{} trees'.format(s))

Давайте рассмотрим описанное явление на более реалистичном наборе данных.

In [13]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

ds = datasets.load_diabetes()
X = ds.data
Y = ds.target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.5, test_size=0.5)

In [ ]:
MAX_ESTIMATORS = 300

gbclf = BaggingRegressor(warm_start=True)
err_train_bag = []
err_test_bag = []
for i in range(1, MAX_ESTIMATORS+1):
    gbclf.n_estimators = i
    gbclf.fit(X_train, Y_train)
    err_train_bag.append(1 - gbclf.score(X_train, Y_train))
    err_test_bag.append(1 - gbclf.score(X_test, Y_test))
    
gbclf = GradientBoostingRegressor(warm_start=True, max_depth=2, learning_rate=0.1)
err_train_gb = []
err_test_gb = []
for i in range(1, MAX_ESTIMATORS+1):
    gbclf.n_estimators = i
    gbclf.fit(X_train, Y_train)
    err_train_gb.append(1 - gbclf.score(X_train, Y_train))
    err_test_gb.append(1 - gbclf.score(X_test, Y_test))
    
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(err_train_gb, label='GB')
plt.plot(err_train_bag, label='Bagging')
plt.legend()
plt.title('Train')
plt.subplot(1, 2, 2)
plt.plot(err_test_gb, label='GB')
plt.plot(err_test_bag, label='Bagging')
plt.legend()
plt.title('Test')
plt.gcf().set_size_inches(15,7)

## Многоклассовая классификация с использованием деревьев решений, случайных лесов и градиентного бустинга

Применим каждый метод к задаче классификации и выберем оптимальную модель.

In [15]:
from sklearn.datasets import load_digits

data = load_digits()
X = data.data
y = data.target

Мы будем использовать набор данных digits. Это задача распознавания рукописных цифр — многозначная классификация по 10 классам.

In [ ]:
np.unique(y)

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(6, 6))
fig.suptitle("Training data examples")

for i in range(9):
    img = X[i].reshape(8, 8)
    axs[i // 3, i % 3].imshow(img)
    axs[i // 3, i % 3].set_title("Class label: %s" % y[i])
    axs[i // 3, i % 3].axis("off")

Разделите набор данных, чтобы иметь возможность проверить свою модель, используйте `train_test_split`.

In [ ]:
# Split the dataset. Use any method you prefer

#### Decision trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Create and fit decision tree with the default parameters
# Evaluate it on the validation set. Use accuracy

#### Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create RandomForestClassifier with the default parameters
# Fit and evaluate

In [ ]:
# Now let's see how the quality depends on the number of models in the ensemble
# For each value in [5, 10, 100, 500, 1000] create a random forest with the corresponding size, fit a model and evaluate
# How does the quality change? What number is sufficient?
# Please write you conslusions

#### Gradient boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Create GradientBoostingClassifier with the default parameters
# Fit and evaluate. Compare its quality to random forest

In [ ]:
# Now let's see how the quality depends on the number of models in the ensemble
# For each value in [5, 10, 100, 500, 1000] train a gradient boosting with the corresponding size
# How does the quality change? What number is sufficient?
# Please write you conslusions

## XGBoost

[Оригинальная статья](https://arxiv.org/pdf/1603.02754.pdf):

1. Распараллеливание, оптимизация и поддержка разреженных/отсутствующих данных

2. Базовый алгоритм аппроксимирует направление, вычисленное с использованием производных функции потерь второго порядка.

3. Регуляризированная цель обучения: добавлены штраф за количество листьев и нормы коэффициентов.

4. Предложен Weighted Quantile Sketch алгоритм для выбора точек разделения узлов дерева.

##### Установка

http://xgboost.readthedocs.io/en/latest/build.html

или

```pip install xgboost```

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(<TRAIN DATA>, label=<TRAIN LABELS>)
dtest = xgb.DMatrix(<TEST DATA>, label=<TEST LABELS>)

param = {
    'max_depth': 3,  # max tree depth
    'eta': 0.3,  # learning rate / step size
    'silent': 1,  # log verbosity
    'objective': 'multi:softprob',  # objective function (in this case for multiclass classification task)
    'num_class': 10}  # number of classes
num_round = 20  # boosting iteration num

bst = xgb.train(param, dtrain, num_round)

preds = bst.predict(dtest)
best_preds = np.asarray([np.argmax(line) for line in preds])

In [ ]:
print(f'accuracy score: {accuracy_score(y_test, best_preds):.4f}')